In [1]:
import pandas as pd
import numpy as np 
import requests
from time import sleep
from urllib.parse import urlparse
import os
import re

In [2]:
from urllib.parse import urlparse, parse_qs
from concurrent.futures import ThreadPoolExecutor

In [3]:
"""
Link to API:
https://api.inaturalist.org/v1/docs/#!/Observations/get_observations

API limits:
Max. 100 request per minute, best 60 requests per minute,
max 10.000 request per day

"""

# List of places to download
PLACE_IDS_NAMES = {
    54321: 'San_Francisco_Bay',

}

def download_all_places(place_ids_names):
    """ Download observations for all specified places """

    for place_id, save_name in PLACE_IDS_NAMES.items():
        download_observations(place_id, save_name)


def download_observations(place_id, save_name, save_path='/Users/zahrafarook/Desktop/DataAnalysis/data/'):
    
    """ Downloads and saves all observations for a specific project/place.

    Each city nature challenge in one year and city is one project.
    Observations have to be downloaded in 200 chunks. (There is also a limit
    at 10.000 observations, which is why the ID filter in the request
    has to be increased.)

    Arguments:
    ----------
     - project_id: int or str
        The id of the project. One can find it on iNaturalist by searching for
        and selecting a project, and then checking the active filter (to the
        right) which shows the ID of the selected project.

     - save_name: str
        The name of the saved file, should be the name of the project
        and year (convention for example "CNC London 2018").

     - save_path: str
        Relative path where files are saved.
    """
    year = 2016 #change the year
    
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
        
    data = []
    id_above = 1
    num_total_obs = None

    print("\nStarting download")
    print(f"project id: {place_id}")
    print(f"save name: {save_name}{year}\n")

    while True:
        # Update req string
        #req_str = f"https://api.inaturalist.org/v1/observations?place_id={place_id}&per_page=200&id_above={id_above}&order=asc&order_by=id"
        #https://www.inaturalist.org/observations?d1=2016-01-01&d2=2016-12-31&place_id=54321&subview=map
        
        req_str = f"https://api.inaturalist.org/v1/observations?d1={start_date}&d2={end_date}&place_id={place_id}&per_page=200&id_above={id_above}&order=asc&order_by=id"

        # Download results
        res = requests.get(req_str).json()
        results = res['results']

        # check total number of results (only correct at first request)
        if num_total_obs is None:
            num_total_obs = res['total_results']

        # If no results left, stop download
        no_more_results = len(results) == 0
        if no_more_results:
            break

        # Save data for each result
        for result in results:
            # Some data processing of location
            if result['location']:
                coords = list(result['location'].split(','))
                lon = coords[1]
                lat = coords[0]
            else:
                lon = None
                lat = None

            # All data that is to be saved
            data_result = {
                'id': result['id'],
                'Joined':result['joined_date']
                'observed_on_string': result['observed_on_string'],
                'observed_on': result['observed_on'],
                'time_observed_at': result['time_observed_at'],
                'created_time_zone': result['created_time_zone'],
                'created_at': result['created_at'],
                'updated_at': result['updated_at'],
                'description': result['description'],
                'user_id': result['user']['id'],
                #'user_login': result['user']['login'] //removing for privacy 
                'quality_grade': result['quality_grade'],
                'reviewed_by': result['reviewed_by'],
                'faves_count': result['faves_count'],
                'num_identification_agreements': result["num_identification_agreements"],
                'num_identification_disagreements': result['num_identification_disagreements'],
                'identifications_most_agree': result['identifications_most_agree'],
                'identifications_most_disagree': result['identifications_most_disagree'],
                'captive': result['captive'],
                'place_guess': result['place_guess'],
                'place_ids': result['place_ids'],
                'longitude': lon,
                'latitude': lat,
                'positional_accuracy': result['positional_accuracy'],
                'geoprivacy': result['geoprivacy'],
                'taxon_geoprivacy': result['taxon_geoprivacy'],
                'obscured': result['obscured'],
                'species_guess': result['species_guess'],
            }
            # Additional taxon information if available
            if result['taxon']:
                taxon_info = {
                    'taxon_id': result['taxon']['id'],
                    'taxon_name': result['taxon']['name'],
                    'preferred_common_name': result['taxon']['preferred_common_name'] if 'preferred_common_name' in result['taxon'].keys() else None,
                    'iconic_taxon_name': result['taxon']['iconic_taxon_name'] if 'iconic_taxon_name' in result['taxon'].keys() else None,
                    'taxon_rank': result['taxon']['rank'],
                    'taxon_parent_id': result['taxon']['parent_id'],
                    'taxon_native': result['taxon']['native'],
                    'taxon_endemic': result['taxon']['endemic'],
                    'taxon_threatened': result['taxon']['threatened'],
                    'taxon_search_rank': result['taxon']['universal_search_rank'],
                    'taxon_observations': result['taxon']['observations_count'],
                }
                data_result = {**data_result, **taxon_info}

            # Identifications: Who identified which user
            if result['identifications']:
                identifications = []
                for identification in result['identifications']:
                    identifications.append({
                        'user_id': identification['user']['id'],
                        'category': identification["category"],
                        'disagreement': identification["disagreement"],
                    })
                data_result['identifications'] = identifications
            data.append(data_result)

        # Log some info on the download progress
        num_obs_downloaded = len(data)
        percent_done = num_obs_downloaded / num_total_obs
        print(
            f" - downloaded {percent_done:.1%} ({num_obs_downloaded}/{num_total_obs} observations)")

        # Get highest ID for where to start next request
        id_above = results[-1]['id']

        # Sleep to leave API some rest
        sleep(1)

    # Convert to df
    df = pd.DataFrame(data)
    df = df.replace(np.nan, None).convert_dtypes()  # infer best dype

    # Save df
    save_filepath = f"{save_path}{save_name}_{year}.csv"
    print("Saving file ", save_filepath)
    df.to_csv(save_filepath, index=False)

    print("Done\n")
    

if __name__ == '__main__':
    for place_id, save_name in PLACE_IDS_NAMES.items():
        download_observations(place_id,save_name)




Starting download
project id: 54321
save name: San_Francisco_Bay2023

 - downloaded 0.0% (200/826973 observations)
 - downloaded 0.0% (400/826973 observations)
 - downloaded 0.1% (600/826973 observations)
 - downloaded 0.1% (800/826973 observations)
 - downloaded 0.1% (1000/826973 observations)
 - downloaded 0.1% (1200/826973 observations)
 - downloaded 0.2% (1400/826973 observations)
 - downloaded 0.2% (1600/826973 observations)
 - downloaded 0.2% (1800/826973 observations)
 - downloaded 0.2% (2000/826973 observations)
 - downloaded 0.3% (2200/826973 observations)
 - downloaded 0.3% (2400/826973 observations)
 - downloaded 0.3% (2600/826973 observations)
 - downloaded 0.3% (2800/826973 observations)
 - downloaded 0.4% (3000/826973 observations)
 - downloaded 0.4% (3200/826973 observations)
 - downloaded 0.4% (3400/826973 observations)
 - downloaded 0.4% (3600/826973 observations)
 - downloaded 0.5% (3800/826973 observations)
 - downloaded 0.5% (4000/826973 observations)
 - downloaded

 - downloaded 4.2% (35000/826973 observations)
 - downloaded 4.3% (35200/826973 observations)
 - downloaded 4.3% (35400/826973 observations)
 - downloaded 4.3% (35600/826973 observations)
 - downloaded 4.3% (35800/826973 observations)
 - downloaded 4.4% (36000/826973 observations)
 - downloaded 4.4% (36200/826973 observations)
 - downloaded 4.4% (36400/826973 observations)
 - downloaded 4.4% (36600/826973 observations)
 - downloaded 4.4% (36800/826973 observations)
 - downloaded 4.5% (37000/826973 observations)
 - downloaded 4.5% (37200/826973 observations)
 - downloaded 4.5% (37400/826973 observations)
 - downloaded 4.5% (37600/826973 observations)
 - downloaded 4.6% (37800/826973 observations)
 - downloaded 4.6% (38000/826973 observations)
 - downloaded 4.6% (38200/826973 observations)
 - downloaded 4.6% (38400/826973 observations)
 - downloaded 4.7% (38600/826973 observations)
 - downloaded 4.7% (38800/826973 observations)
 - downloaded 4.7% (39000/826973 observations)
 - downloaded

 - downloaded 8.5% (70000/826973 observations)
 - downloaded 8.5% (70200/826973 observations)
 - downloaded 8.5% (70400/826973 observations)
 - downloaded 8.5% (70600/826973 observations)
 - downloaded 8.6% (70800/826973 observations)
 - downloaded 8.6% (71000/826973 observations)
 - downloaded 8.6% (71200/826973 observations)
 - downloaded 8.6% (71400/826973 observations)
 - downloaded 8.7% (71600/826973 observations)
 - downloaded 8.7% (71800/826973 observations)
 - downloaded 8.7% (72000/826973 observations)
 - downloaded 8.7% (72200/826973 observations)
 - downloaded 8.8% (72400/826973 observations)
 - downloaded 8.8% (72600/826973 observations)
 - downloaded 8.8% (72800/826973 observations)
 - downloaded 8.8% (73000/826973 observations)
 - downloaded 8.9% (73200/826973 observations)
 - downloaded 8.9% (73400/826973 observations)
 - downloaded 8.9% (73600/826973 observations)
 - downloaded 8.9% (73800/826973 observations)
 - downloaded 8.9% (74000/826973 observations)
 - downloaded

 - downloaded 12.6% (104400/826973 observations)
 - downloaded 12.6% (104600/826973 observations)
 - downloaded 12.7% (104800/826973 observations)
 - downloaded 12.7% (105000/826973 observations)
 - downloaded 12.7% (105200/826973 observations)
 - downloaded 12.7% (105400/826973 observations)
 - downloaded 12.8% (105600/826973 observations)
 - downloaded 12.8% (105800/826973 observations)
 - downloaded 12.8% (106000/826973 observations)
 - downloaded 12.8% (106200/826973 observations)
 - downloaded 12.9% (106400/826973 observations)
 - downloaded 12.9% (106600/826973 observations)
 - downloaded 12.9% (106800/826973 observations)
 - downloaded 12.9% (107000/826973 observations)
 - downloaded 13.0% (107200/826973 observations)
 - downloaded 13.0% (107400/826973 observations)
 - downloaded 13.0% (107600/826973 observations)
 - downloaded 13.0% (107800/826973 observations)
 - downloaded 13.1% (108000/826973 observations)
 - downloaded 13.1% (108200/826973 observations)
 - downloaded 13.1% 

 - downloaded 16.7% (138000/826973 observations)
 - downloaded 16.7% (138200/826973 observations)
 - downloaded 16.7% (138400/826973 observations)
 - downloaded 16.8% (138600/826973 observations)
 - downloaded 16.8% (138800/826973 observations)
 - downloaded 16.8% (139000/826973 observations)
 - downloaded 16.8% (139200/826973 observations)
 - downloaded 16.9% (139400/826973 observations)
 - downloaded 16.9% (139600/826973 observations)
 - downloaded 16.9% (139800/826973 observations)
 - downloaded 16.9% (140000/826973 observations)
 - downloaded 17.0% (140200/826973 observations)
 - downloaded 17.0% (140400/826973 observations)
 - downloaded 17.0% (140600/826973 observations)
 - downloaded 17.0% (140800/826973 observations)
 - downloaded 17.1% (141000/826973 observations)
 - downloaded 17.1% (141200/826973 observations)
 - downloaded 17.1% (141400/826973 observations)
 - downloaded 17.1% (141600/826973 observations)
 - downloaded 17.1% (141800/826973 observations)
 - downloaded 17.2% 

...

 - downloaded 81.7% (675600/826973 observations)
 - downloaded 81.7% (675800/826973 observations)
 - downloaded 81.7% (676000/826973 observations)
 - downloaded 81.8% (676200/826973 observations)
 - downloaded 81.8% (676400/826973 observations)
 - downloaded 81.8% (676600/826973 observations)
 - downloaded 81.8% (676800/826973 observations)
 - downloaded 81.9% (677000/826973 observations)
 - downloaded 81.9% (677200/826973 observations)
 - downloaded 81.9% (677400/826973 observations)
 - downloaded 81.9% (677600/826973 observations)
 - downloaded 82.0% (677800/826973 observations)
 - downloaded 82.0% (678000/826973 observations)
 - downloaded 82.0% (678200/826973 observations)
 - downloaded 82.0% (678400/826973 observations)
 - downloaded 82.1% (678600/826973 observations)
 - downloaded 82.1% (678800/826973 observations)
 - downloaded 82.1% (679000/826973 observations)
 - downloaded 82.1% (679200/826973 observations)
 - downloaded 82.2% (679400/826973 observations)
 - downloaded 82.2% 

 - downloaded 85.8% (709200/826973 observations)
 - downloaded 85.8% (709400/826973 observations)
 - downloaded 85.8% (709600/826973 observations)
 - downloaded 85.8% (709800/826973 observations)
 - downloaded 85.9% (710000/826973 observations)
 - downloaded 85.9% (710200/826973 observations)
 - downloaded 85.9% (710400/826973 observations)
 - downloaded 85.9% (710600/826973 observations)
 - downloaded 86.0% (710800/826973 observations)
 - downloaded 86.0% (711000/826973 observations)
 - downloaded 86.0% (711200/826973 observations)
 - downloaded 86.0% (711400/826973 observations)
 - downloaded 86.0% (711600/826973 observations)
 - downloaded 86.1% (711800/826973 observations)
 - downloaded 86.1% (712000/826973 observations)
 - downloaded 86.1% (712200/826973 observations)
 - downloaded 86.1% (712400/826973 observations)
 - downloaded 86.2% (712600/826973 observations)
 - downloaded 86.2% (712800/826973 observations)
 - downloaded 86.2% (713000/826973 observations)
 - downloaded 86.2% 

 - downloaded 89.8% (742800/826973 observations)
 - downloaded 89.8% (743000/826973 observations)
 - downloaded 89.9% (743200/826973 observations)
 - downloaded 89.9% (743400/826973 observations)
 - downloaded 89.9% (743600/826973 observations)
 - downloaded 89.9% (743800/826973 observations)
 - downloaded 90.0% (744000/826973 observations)
 - downloaded 90.0% (744200/826973 observations)
 - downloaded 90.0% (744400/826973 observations)
 - downloaded 90.0% (744600/826973 observations)
 - downloaded 90.1% (744800/826973 observations)
 - downloaded 90.1% (745000/826973 observations)
 - downloaded 90.1% (745200/826973 observations)
 - downloaded 90.1% (745400/826973 observations)
 - downloaded 90.2% (745600/826973 observations)
 - downloaded 90.2% (745800/826973 observations)
 - downloaded 90.2% (746000/826973 observations)
 - downloaded 90.2% (746200/826973 observations)
 - downloaded 90.3% (746400/826973 observations)
 - downloaded 90.3% (746600/826973 observations)
 - downloaded 90.3% 

 - downloaded 93.9% (776400/826973 observations)
 - downloaded 93.9% (776600/826973 observations)
 - downloaded 93.9% (776800/826973 observations)
 - downloaded 94.0% (777000/826973 observations)
 - downloaded 94.0% (777200/826973 observations)
 - downloaded 94.0% (777400/826973 observations)
 - downloaded 94.0% (777600/826973 observations)
 - downloaded 94.1% (777800/826973 observations)
 - downloaded 94.1% (778000/826973 observations)
 - downloaded 94.1% (778200/826973 observations)
 - downloaded 94.1% (778400/826973 observations)
 - downloaded 94.2% (778600/826973 observations)
 - downloaded 94.2% (778800/826973 observations)
 - downloaded 94.2% (779000/826973 observations)
 - downloaded 94.2% (779200/826973 observations)
 - downloaded 94.2% (779400/826973 observations)
 - downloaded 94.3% (779600/826973 observations)
 - downloaded 94.3% (779800/826973 observations)
 - downloaded 94.3% (780000/826973 observations)
 - downloaded 94.3% (780200/826973 observations)
 - downloaded 94.4% 

 - downloaded 97.9% (810000/826973 observations)
 - downloaded 98.0% (810200/826973 observations)
 - downloaded 98.0% (810400/826973 observations)
 - downloaded 98.0% (810600/826973 observations)
 - downloaded 98.0% (810800/826973 observations)
 - downloaded 98.1% (811000/826973 observations)
 - downloaded 98.1% (811200/826973 observations)
 - downloaded 98.1% (811400/826973 observations)
 - downloaded 98.1% (811600/826973 observations)
 - downloaded 98.2% (811800/826973 observations)
 - downloaded 98.2% (812000/826973 observations)
 - downloaded 98.2% (812200/826973 observations)
 - downloaded 98.2% (812400/826973 observations)
 - downloaded 98.3% (812600/826973 observations)
 - downloaded 98.3% (812800/826973 observations)
 - downloaded 98.3% (813000/826973 observations)
 - downloaded 98.3% (813200/826973 observations)
 - downloaded 98.4% (813400/826973 observations)
 - downloaded 98.4% (813600/826973 observations)
 - downloaded 98.4% (813800/826973 observations)
 - downloaded 98.4% 